# LLM 입력용 데이터 생성

In [98]:
import os, sys
from tqdm import tqdm
sys.path.append(os.path.abspath(".."))
from functions import *
from transformers import ElectraModel, ElectraTokenizer

## 1. 데이터 읽어오기

In [2]:
save_path = "../data/target"
data_path = "../data/kipris_ami"

In [65]:
for filename in tqdm(os.listdir(data_path)):
    if "claim" in filename:
        claim_df = read_csv(os.path.join(data_path, filename))
    else:
        main_df = read_csv(os.path.join(data_path, filename))

len(claim_df), len(main_df)

 50%|█████     | 1/2 [02:22<02:22, 142.80s/it]/home/hee/projects/patent/functions.py:10: DtypeWarning: Columns (10,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
100%|██████████| 2/2 [03:26<00:00, 103.44s/it]


(2640867, 2634934)

In [79]:
claim_df = claim_df.drop(columns=['Unnamed: 0'])
main_df = main_df.drop(columns=['Unnamed: 0'])

In [66]:
claim_df_original = claim_df.copy()

## 2. G섹션 데이터 통계내고, 섹션 선택

In [5]:
# cpc_dict = {}
# errorData = []

# for _, data in tqdm(main_df.iterrows(), total=len(main_df)):
#     try:
#         cpc_str = data['listCPC']  # None값인 경우도 있음
#         cpc_str = cpc_str.replace("[", ""); cpc_str = cpc_str.replace("]", "")
#     except AttributeError:
#         continue
#     except Exception as e:
#         errorData.append([data['applicationNumber'], e])

#     cpc_list = [cpc.strip() for cpc in cpc_str.split(",")]
#     split_cpc_list = [cpc[:4] for cpc in cpc_list]

#     for cpc in set(split_cpc_list):
#         if cpc[0] == 'G':
#             if cpc not in cpc_dict: cpc_dict[cpc] = 1
#             else: cpc_dict[cpc]+=1

In [6]:
# cpc_dict['G16Y']  # G:물리학. 사물 인터넷 [IoT]에 특히 적합한 정보 및 통신 기술

In [7]:
# len(errorData)

## 3. G16Y 섹션 특허 출원번호 추출

In [8]:
target_patent_number = []
errorData = []

for _, data in tqdm(main_df.iterrows(), total=len(main_df)):
    try:
        cpc_str = data['listCPC']  # None값인 경우도 있음
        cpc_str = cpc_str.replace("[", ""); cpc_str = cpc_str.replace("]", "")
    except AttributeError:
        continue
    except Exception as e:
        errorData.append([data['applicationNumber'], e])

    cpc_list = [cpc.strip() for cpc in cpc_str.split(",")]
    split_cpc_list = [cpc[:4] for cpc in cpc_list]

    for cpc in set(split_cpc_list):
        if cpc[:4] == "G16Y":
            patentNumber = data['applicationNumber']
            target_patent_number.append(patentNumber)
            break

100%|██████████| 2634934/2634934 [03:33<00:00, 12338.45it/s]


In [9]:
# claim_df = claim_df.drop(columns=['Unnamed: 0'])
# claim_df.head()

In [80]:
claim_df = claim_df[claim_df['applicationNumber'].isin(target_patent_number)]
claim_df = claim_df.reset_index(drop=True)  
claim_df.head()

,seq,applicationNumber,claim
0,653,1020190007311,1. 다수곳에 배치된 세정기들에 개별적으로 설치된 통신모듈 및 사용자의 정보를 공유...
1,679,1020190007627,1. 블록체인에 기반한 스마트 팩토리 구현을 위한 인공지능 IIoT(Industri...
2,1333,1020190016713,"1. 태양전지판과, 상기 태양전지판에서 생산한 전력을 저장하는 배터리와, 상기 태양..."
3,1448,1020190019082,"1. 실시간 메이크업 진단 및 질의응답 서비스 제공 시스템으로서,유저로부터 유저정보..."
4,1487,1020190019837,1. 연속된 단계의 반도체 공정들로 이루어진 제조 설비 시스템의 분산처리 공정관리를...


## 4. 청구항 건수 카운팅
- 문장 개수, 길이 조절

In [11]:
# claim_row_count_dict = {}

# for _, row in tqdm(claim_df.iterrows(), total=len(claim_df)):
#     original_claim = row['claim']
#     row_claim_dict = {}
#     split_claim = original_claim.split("\n")
#     row_claim_dict['문장개수'] = len(split_claim)
#     row_claim_dict['문장길이'] = []

#     for claim in split_claim:
#         if len(claim)!=0:
#             row_claim_dict['문장길이'].append(len(claim))
#     claim_row_count_dict[row['applicationNumber']] = row_claim_dict
    

In [12]:
# claim_row_count_dict

In [13]:
# claim_df.to_csv(os.path.join(save_path, "G16Y_118.csv"), encoding='euc-kr', index=False)

# 4. 일단 문장만 처리

In [81]:
# 불용어 제거
def delete_word(text: str)->str:
    delete_word = ["삭제", "<P INDENT='14' ALIGN='JUSTIFIED'>", "</P>", "\n"]
    for delete in delete_word:
        text = text.replace(delete, "", -1)
    return text

claim_df['claim'] = claim_df['claim'].apply(lambda data: delete_word(data))
claim_df

,seq,applicationNumber,claim
0,653,1020190007311,1. 다수곳에 배치된 세정기들에 개별적으로 설치된 통신모듈 및 사용자의 정보를 공유...
1,679,1020190007627,1. 블록체인에 기반한 스마트 팩토리 구현을 위한 인공지능 IIoT(Industri...
2,1333,1020190016713,"1. 태양전지판과, 상기 태양전지판에서 생산한 전력을 저장하는 배터리와, 상기 태양..."
3,1448,1020190019082,"1. 실시간 메이크업 진단 및 질의응답 서비스 제공 시스템으로서,유저로부터 유저정보..."
4,1487,1020190019837,1. 연속된 단계의 반도체 공정들로 이루어진 제조 설비 시스템의 분산처리 공정관리를...
...,...,...,...
113,1586183,1020060015085,"1. 홈 네트워크의 홈 서버에 연결되는 가전기기와, 상기 가전기기와 이동통신단말기를..."
114,1691799,1020060135481,1. 발신측 이동단말기에 입력된 사용자의 음성신호를 상기 발신측 이동단말기의 번호와...
115,1870812,1020040058953,1. 복합기(1)에서 그 일측에 인공지능컴퓨터프로세서와 칩이 내장된 컴퓨터기(23)...
116,1870821,1020040058962,1. 집(1)에서 그 일측에 인공지능컴퓨터프로세서와 칩이 내장된 컴퓨터기(23)에 ...


In [88]:
#TODO: functions.py의 delete_claim_number 함수로 교체
def text_replace(text:str)->str:
    new_text = re.sub(r"(\d)\. ", " ", text, count=1)
    new_text = re.sub(r"(\d)\. ", "\n", new_text)
    return new_text.strip()

claim_df['claim'] = claim_df['claim'].apply(lambda data: text_replace(data))
claim_df

,seq,applicationNumber,claim
0,653,1020190007311,다수곳에 배치된 세정기들에 개별적으로 설치된 통신모듈 및 사용자의 정보를 공유하는 ...
1,679,1020190007627,블록체인에 기반한 스마트 팩토리 구현을 위한 인공지능 IIoT(Industrial ...
2,1333,1020190016713,"태양전지판과, 상기 태양전지판에서 생산한 전력을 저장하는 배터리와, 상기 태양전지판..."
3,1448,1020190019082,"실시간 메이크업 진단 및 질의응답 서비스 제공 시스템으로서,유저로부터 유저정보를 입..."
4,1487,1020190019837,연속된 단계의 반도체 공정들로 이루어진 제조 설비 시스템의 분산처리 공정관리를 위해...
...,...,...,...
113,1586183,1020060015085,"홈 네트워크의 홈 서버에 연결되는 가전기기와, 상기 가전기기와 이동통신단말기를 홈 ..."
114,1691799,1020060135481,발신측 이동단말기에 입력된 사용자의 음성신호를 상기 발신측 이동단말기의 번호와 맵핑...
115,1870812,1020040058953,복합기(1)에서 그 일측에 인공지능컴퓨터프로세서와 칩이 내장된 컴퓨터기(23)에 의...
116,1870821,1020040058962,집(1)에서 그 일측에 인공지능컴퓨터프로세서와 칩이 내장된 컴퓨터기(23)에 의해 ...


In [89]:
claim_df2 = claim_df.copy()
claim_df2 = claim_df2.assign(sentence=claim_df2['claim'].str.split("\n")).explode("sentence").reset_index(drop=True) # 청구항 문장 단위로 분리
claim_df2 = claim_df2[claim_df2['sentence'].apply(lambda data: True if len(data)>1 else False)]  # 공백 제거
claim_df2 = claim_df2.assign(len_sen=claim_df2['sentence'].apply(lambda data: len(data)))
claim_df2 = claim_df2.drop(columns=['claim'])
claim_df2

,seq,applicationNumber,sentence,len_sen
0,653,1020190007311,다수곳에 배치된 세정기들에 개별적으로 설치된 통신모듈 및 사용자의 정보를 공유하는 ...,360
1,653,1020190007311,"제 1 항에 있어서,상기 사용자의 단말기에 애플리케이션을 설치하는 단계에서는 설치된...",161
2,679,1020190007627,블록체인에 기반한 스마트 팩토리 구현을 위한 인공지능 IIoT(Industrial ...,5098
3,1333,1020190016713,"태양전지판과, 상기 태양전지판에서 생산한 전력을 저장하는 배터리와, 상기 태양전지판...",1150
5,1333,1020190016713,"제1항에 있어서,상기 메인 컨트롤러는상기 IoT 모듈로부터 전송되는 수질 측정신호를...",614
...,...,...,...,...
1101,1909225,1020040101224,"제2항에 있어서, 상기 멀티...",240
1102,1909225,1020040101224,"제2항에 있어서, 상기 콘솔...",257
1103,1909225,1020040101224,"제5항에 있어서, 상기 콘솔...",288
1104,1909225,1020040101224,"제5항에 있어서, 상기 서비...",245


In [ ]:
# claim_df2.to_csv(os.path.join(save_path, "G16Y_118_v4.csv"), encoding='euc-kr', index=False)

In [87]:
len(claim_df2['applicationNumber'].unique())

118

In [96]:
target_df = claim_df2[(claim_df2['len_sen'] >= 194) & (claim_df2['len_sen'] <= 294)]
len(target_df)

183

# 5. 문장 토크나이징

# 특허 데이터 확인

In [102]:
#TODO: functions.py 수정

model_path = "../KIPI-KorPatELECTRA/KorPatELECTRA/PT"
tokenizer = ElectraTokenizer.from_pretrained(model_path)
def tokenizing(sen: str)-> str:
    tokens = tokenizer.tokenize(sen)
    tokens = ["[CLS]"] + tokens + ["[SEP]"]

    return ", ".join(token for token in tokens)
    
target_df['tokens'] = target_df['sentence'].apply(lambda data: tokenizing(data))
target_df

/tmp/ipykernel_143817/2279253133.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_df['tokens'] = target_df['sentence'].apply(lambda data: tokenizing(data))


,seq,applicationNumber,sentence,len_sen,tokens
8,1448,1020190019082,"제 1항에 있어서,상기 시스템은,상기 촬영이미지에 대한 가이드이미지를 생성하는 가이...",231,"[CLS], 제, 1, ##항, ##에, 있, ##어서, ,, 상기, 시스템, ##..."
9,1448,1020190019082,"제 2항에 있어서,상기 시스템은,상기 촬영이미지와 상기 가이드이미지의 각각의 구도를...",275,"[CLS], 제, 2, ##항, ##에, 있, ##어서, ,, 상기, 시스템, ##..."
25,2451,1020190043955,수신된 디바이스 정보를 디바이스 등록 정보로 저장하는 디바이스 등록부;새로운 디바이...,277,"[CLS], 수신, ##된, 디바이스, 정보, ##를, 디바이스, 등록, 정보, #..."
27,2451,1020190043955,"제 1 항에 있어서,상기 디바이스 접속 결정부는,상기 접속 요청 신호를 수신하는 경...",195,"[CLS], 제, 1, 항, ##에, 있, ##어서, ,, 상기, 디바이스, 접속,..."
36,3672,1020180000048,"제1항에 있어서,모듈형 컨테이너 부스의 상부에 형성된 전광판은 2단 이상의 복층으로...",194,"[CLS], 제, ##1, ##항, ##에, 있, ##어서, ,, 모듈, ##형, ..."
...,...,...,...,...,...
1101,1909225,1020040101224,"제2항에 있어서, 상기 멀티...",240,"[CLS], 제, ##2, ##항, ##에, 있, ##어서, ,, 상기, 멀티, 콘..."
1102,1909225,1020040101224,"제2항에 있어서, 상기 콘솔...",257,"[CLS], 제, ##2, ##항, ##에, 있, ##어서, ,, 상기, 콘솔, #..."
1103,1909225,1020040101224,"제5항에 있어서, 상기 콘솔...",288,"[CLS], 제, ##5, ##항, ##에, 있, ##어서, ,, 상기, 콘솔, #..."
1104,1909225,1020040101224,"제5항에 있어서, 상기 서비...",245,"[CLS], 제, ##5, ##항, ##에, 있, ##어서, ,, 상기, 서비스, ..."


In [104]:
target_df.to_csv(os.path.join(save_path, "llm프롬포팅용데이터.csv"), encoding='euc-kr', index=False)

In [64]:
for claim in claim_df[claim_df['applicationNumber']==1020190019082]['claim'].str.split("\n"):
    print(claim)

['실시간 메이크업 진단 및 질의응답 서비스 제공 시스템으로서,유저로부터 유저정보를 입력받는 유저가입부 및, 전문가로부터 전문가정보를 입력받는 전문가가입부를 포함하는 가입처리모듈;복수의 상기 전문가를 리스트화한 멘토리스트를 생성하는 리스트생성부 및, 상기 멘토리스트를 상기 유저에게 제공하는 리스트제공부와, 상기 유저로부터 상기 멘토리스트에 속한 상기 전문가 중 적어도 어느 하나를 선택받는 멘토선택부를 포함하는 멘토선택모듈;상기 유저로부터 상기 유저의 안면부를 촬영한 적어도 하나 이상의 촬영이미지 및 관련질문을 입력받는 질문입력모듈;상기 유저로부터 선택받은 상기 전문가에게 상기 유저의 촬영이미지 및 상기 관련질문을 제공하는 질문제공모듈;상기 촬영이미지를 분석하여 상기 촬영이미지로부터 적어도 하나 이상의 메이크업영역을 검출하는 영역검출부 및, 각각의 상기 메이크업영역에 대해 색상정보 및 도포정보를 생성하는 영역분석부와, 상기 색상정보 및 상기 도포정보에 따라 각각의 상기 메이크업영역에 대한 영역별점수를 산출하는 영역별산출부 및, 상기 영역별점수를 기반으로 하여 상기 촬영이미지에 대한 종합점수를 산출하는 종합점수산출부를 포함하는 자동분석모듈;상기 전문가로부터 멘토코멘트를 입력받는 코멘트입력부 및, 상기 멘토코멘트 및 상기 영역별점수와 상기 종합점수를 상기 유저에게 제공하는 코멘트전달부를 포함하는 멘토링수행모듈;을 포함하고, 상기 질문입력모듈은, 상기 유저에게 기 설정된 기준색상이미지를 제공하는 기준색상제공부 및, 상기 유저로부터 상기 기준색상이미지 및 상기 유저의 안면부가 함께 촬영된 적어도 하나 이상의 촬영이미지와 상기 관련질문을 입력받는 질문입력부를 포함하며,상기 자동분석모듈은,유저에게 제공된 상기 기준색상이미지의 색상정보인 기준색상정보를 분석하는 기준분석파트 및, 상기 촬영이미지에 포함된 상기 기준색상이미지의 색상정보인 입력색상정보를 분석하는 비교분석파트와, 상기 입력색상정보와 상기 기준색상정보가 일치되도록 상기 촬영이미지를 보정 처리하는 보정처리파트를 포함하

In [76]:
# 1020190019082
# 1020090108319
# 1020040058962

target = claim_df_original[claim_df_original['applicationNumber']==1020040058962]

In [77]:
for claim in target['claim'].str.split("\n"):
    print(claim)

["1. <P INDENT='14' ALIGN='JUSTIFIED'>집(1)에서 그 일측에 인공지능컴퓨터프로세서와 칩이 내장된 컴퓨터기(23)에 의해 입력정보에 따라 움직이는 통제회로부(10)와 전원부(9)와 스윗치(9')와 CCTV 송수신부(15)를 설치구성하여 범인(침입자)가 들어오거나 문앞에 서성될때 이를감지하는 전자감응장치기(2)와 적외선감지기(3)를 문밖이나 문내측 출입구 또는 본발명품내외측에 설치구성하여 놓고 여기서 감지포착된 전기적신호가 통제부(10)로 통보하면 위의 출입구 등,복도에 설치된 CCTV카메라기(15'&quot;)(15')와 본발명품, 본체(22)에 설치된 CCTV카메라기(15&quot;)가 송수신부(15)에 의해 작동 촬영시작되어 실시간으로 동영상, 사진 등을 원격 송수신기(13)에 의해 경찰서 112지령실 또는 사용자등의 휴대폰으로 메모리된 내용이 전송된다 그리고 그 전기적신호에 의해 본발명품을 강제로 열거나 충격을 주거나, 터치시에도 문등의 내측 일측에 설치된 감지센스(2')와 자동 감지센스(2,3)에 의하여 마취탄장치기(4)에 의해 마취탄(4')이 발사부(5)로 발사되어 범인을 항거불능으로 만들어서 체포시키며 또한 동시에 에어백식으로 발사되는 그물발사기(6)를 일측에 설치구성시키며 인공지능컴퓨터(8)가 사람의 음성, 얼굴 등의 인식하여 통제 작동명령을 내려서 위의 발사부(5)가 작동되어 마취탄(4')이나 그외 가스탄(4&quot;)이 발사되어 터지고 그물발사기(6)가 작동는 것을 특징으로 하는 테러방지용 최첨단 하이테크 디지털 기능성의 집 의 제조방법</P>", "2. <P INDENT='14' ALIGN='JUSTIFIED'>제1항에 있어서 습도조절기(11&quot;)가 설치된 것을 특징으로 하는 테러방지용 <DP N='13' TYPE='SOFT'>최첨단 하이테크디지털기능성의 집 의 제조방법</P>", "3. <P INDENT='14' ALIGN='JUSTIFIED'>제1항에 있어서 전자충격기(19)가 설치된 것을 특징으로하는

In [ ]:
#  '11. 삭제', '12. 삭제', '13. 삭제', '14. 삭제', ''